## 문서 분류 모델 훑어보기

### 모델 구조
1. CLS와SEP를 각각 원래 토큰 시퀀스 앞뒤에 붙임
2. BERT 모델에 입력+ 문장 수준의 벡터(pooler_output)를 뽑는다. 
3. 벡터에 추가 모듈을 붙여 \[긍정,부정\]형태

### 태스크 모듈
1. pooler_output에 드롭아웃(3장)을 적용
2. pooler_output을 분류해야할 범주 수만큼의 차원을 갖는 벸터로 변환
3. 소프트 맥스 함수 적용
4. 모델의 출력과 레이블을 같아지도록 BERT레이어를 포함한 모델 전체 업데이트(파인튜닝)

## 문서 분류 모델 학습하기

### 영화리뷰 감성 분석 모델 만들기

##### 기본 설정

In [4]:
# #TPU관련 패키지 설치
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [5]:
!pip install ratsnlp

In [6]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


#### 모델 환경설정

In [7]:
#모델 환경설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base", 
    #프리트레인을 마친 언어 모델이름(허깅패이스 등록된것중)
    downstream_corpus_name="nsmc",
    #다운스림 데이터 이름
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    #파인튜딩된 모델의 체크포인트가 저장될 위치
    batch_size=32 if torch.cuda.is_available() else 4,
    #배치 크기: GPU 32 / TPU=4 /
    learning_rate=5e-5,
    #보폭을 얼마로?
    max_seq_length=128,
    #토큰 기준 입력 문장 최대 길이 if 짧으면 [PAD]뒤에 추가됨
    epochs=3,
    # 학습데이터 전체를 n번 반복하여 학습
    tpu_cores=0 if torch.cuda.is_available() else 8,
    #TPU코어 수, GPU라면 0
    seed=7,
    #랜덤시드 None을 입력시 시드 고정하지 않겠다는 뜻
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [8]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)
# args에 지정된 시드로 고정

set seed: 7


In [9]:
#로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)


#### 말뭉치 내려 받기

In [10]:
# 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)
## 말뭉치(Naver Sentiment Movie review Corpus)를 다운

[nsmc] download ratings_train.txt: 14.6MB [00:00, 84.9MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 35.9MB/s]                           


#### 토크나이저 준비하기

In [11]:
#토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

#### 데이터 전처리

ClassificationDataset은 NscmCorpos와 토크나이저를 품고 있다.

NscmCorpos는 CSV형식의 NSMC 데이터를 문장(영화리뷰)과 레이블(긍정부정)로 읽어 들인다.

NscmCorpos는 ClassificationDataset이 요구하면 문장과 레이블 제공

ClassificationDataset은 문장과 레이블을 각각 토크나이저로 모델이 학습할 수 있는 형태로 변형함

ClassificationFeatures에는
1. input_id:인덱스로 변환된 토큰 시퀀스
2. attention_mask: 패딩 토큰인지(0) 아닌지(1)
3. token_type_ids:세그먼트 정보
4. label:레이블 정보

In [12]:
#학습 데이터 셋 구축

from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 33.866 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [13]:
train_dataset[100]
# train_dataset[100].input_id
# train_dataset[100].attention_mask
# train_dataset[100].token_type_ids
## BERT 모델의 특징
# train_dataset[100].label

ClassificationFeatures(input_ids=[2, 2005, 4024, 4017, 1293, 4599, 4775, 4042, 2478, 4075, 4196, 15, 1463, 4207, 4196, 8080, 4024, 10314, 11219, 4180, 12610, 10579, 832, 4140, 11414, 9827, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
#학습 데이터 로더 구축
## ClassificationDataset 클래스가 들고 있는 인스턴스 중 배치그기만큼을 뽑아 배치 형태로 가공
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    #샘플링 방식: 배치사이즈 개수만큼 전체 인스턴스에서 비복원으로 랜덤추출
    collate_fn=nlpbook.data_collator,
    #뽑은 인스턴스를 배치로 만들기
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [16]:
# 평가용 데이터 구축
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 11.364 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

#### 모델 불러오기

In [21]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

#### 모델 학습시키기

In [22]:
#TASK 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)
#ClassificationTask에는 옵티마이저(아담), 러닝레이트 스케줄러(ExponentialLR)가 정의되 있음


#트레이너 정의
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-doccls exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [23]:
# 학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

##학습 마친 모델을 실전 투입하기

### 영화 리류 감성 분석 웹 서비스 만들기

#### 환경 설정하기

In [ ]:
# !pip install ratsnlp

# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [24]:
#인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    #파인튜닝한 모델의 체크포인트 저장 위치
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


#### 토크나이저 및 모델 불러오기

In [25]:
#체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

#BERT 설정 로드
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

#BERT 모델 초기화
model = BertForSequenceClassification(pretrained_model_config)

#체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#평가 모드로 전환
model.eval()

#토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

#### 모델 출력값 만들고 후처리하기

In [26]:
#인스턴스 함수
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    #문장을 토큰화한 뒤 input_id, attention_masks, token_type_ids 만들기
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})#모델 계산하기
        #{k: torch.tensor(v) for k, v in inputs.items()}: inputs를 파이토치 텐서로 바꾸기
        prob = outputs.logits.softmax(dim=1)#로짓에 소프트맥스 취하기
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        #긍정/부정 확률을 소수점 4자리로 반올림
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
        #예측 확률의 최댓값 위치에 따라 pred 만들기
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [27]:
inference_fn('이딴게 영화 ㅋㅋㅋ')

{'negative_data': '부정 0.9169',
 'negative_width': '91.69%',
 'positive_data': '긍정 0.0831',
 'positive_width': '8.309999999999999%',
 'prediction': '부정 (negative)',
 'sentence': '이딴게 영화 ㅋㅋㅋ'}